# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-13 07:34:05] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=31832, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=71984991, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_

[2025-05-13 07:34:05] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-05-13 07:34:13] Attention backend not set. Use flashinfer backend by default.
[2025-05-13 07:34:13] Init torch distributed begin.


[2025-05-13 07:34:13] Init torch distributed ends. mem usage=0.00 GB
[2025-05-13 07:34:13] Load weight begin. avail mem=62.07 GB


[2025-05-13 07:34:14] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-05-13 07:34:18] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.30s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.30s/it]

[2025-05-13 07:34:20] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=38.62 GB, mem usage=0.93 GB.
[2025-05-13 07:34:20] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-05-13 07:34:20] Memory pool end. avail mem=38.30 GB


[2025-05-13 07:34:20] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096
[2025-05-13 07:34:20] INFO:     Started server process [969047]
[2025-05-13 07:34:20] INFO:     Waiting for application startup.
[2025-05-13 07:34:20] INFO:     Application startup complete.
[2025-05-13 07:34:20] INFO:     Uvicorn running on http://127.0.0.1:31832 (Press CTRL+C to quit)
[2025-05-13 07:34:20] INFO:     127.0.0.1:40602 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-13 07:34:21] INFO:     127.0.0.1:40604 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-13 07:34:21] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-13 07:34:22,452 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

2025-05-13 07:35:09,597 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-13 07:35:09,607 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-05-13 07:35:24,712 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-13 07:35:25] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0
2025-05-13 07:35:25,202 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-05-13 07:35:25,215 - INFO - flashinfer.jit: Finished loading JIT ops: cascade
2025-05-13 07:35:25,269 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-05-13 07:35:39,931 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-05-13 07:35:42,642 - INFO - flashinfer.jit: Loading JIT ops: quantization


2025-05-13 07:35:57,076 - INFO - flashinfer.jit: Finished loading JIT ops: quantization
[2025-05-13 07:35:57] INFO:     127.0.0.1:40616 - "POST /generate HTTP/1.1" 200 OK
[2025-05-13 07:35:57] The server is fired up and ready to roll!


[2025-05-13 07:36:00] INFO:     127.0.0.1:40624 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-05-13 07:36:00] Child process unexpectedly failed with an exit code 9. pid=969622
[2025-05-13 07:36:00] Child process unexpectedly failed with an exit code 9. pid=969441


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-13 07:36:06] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=39087, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1009792696, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu

[2025-05-13 07:36:07] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-05-13 07:36:15] Attention backend not set. Use flashinfer backend by default.
[2025-05-13 07:36:15] Init torch distributed begin.
[2025-05-13 07:36:15] Init torch distributed ends. mem usage=0.00 GB
[2025-05-13 07:36:15] Load weight begin. avail mem=78.58 GB


[2025-05-13 07:36:16] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-05-13 07:36:22] INFO:     127.0.0.1:50188 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-13 07:36:23] INFO:     127.0.0.1:50198 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-13 07:36:23] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-13 07:36:23,852 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-13 07:36:23,880 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-13 07:36:23,888 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-13 07:36:23,901 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-05-13 07:36:24,318 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
2025-05-13 07:36:24,331 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-05-13 07:36:26,755 - INFO - flashinfer.jit: Loading JIT ops: quantization
2025-05-13 07:36:26,769 - INFO - flashinfer.jit: Finished loading JIT ops: quantization
[2025-05-13 07:36:26] INFO:     127.0.0.1:50212 - "POST /generate HTTP/1.1" 200 OK
[2025-05-13 07:36:26] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-05-13 07:36:27] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
2025-05-13 07:36:27,634 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-05-13 07:36:27,647 - INFO - flashinfer.jit: Finished loading JIT ops: cascade


[2025-05-13 07:36:28] INFO:     127.0.0.1:43088 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-05-13 07:36:28] Child process unexpectedly failed with an exit code 9. pid=977074
[2025-05-13 07:36:28] Child process unexpectedly failed with an exit code 9. pid=977008


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-13 07:36:34] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=30335, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=947779467, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None

[2025-05-13 07:36:42] Casting torch.bfloat16 to torch.float16.


[2025-05-13 07:36:42] Casting torch.bfloat16 to torch.float16.
[2025-05-13 07:36:42] Attention backend not set. Use flashinfer backend by default.
[2025-05-13 07:36:42] Init torch distributed begin.


[2025-05-13 07:36:43] Init torch distributed ends. mem usage=0.00 GB
[2025-05-13 07:36:43] Load weight begin. avail mem=78.58 GB


[2025-05-13 07:36:44] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-05-13 07:37:01] INFO:     Started server process [978643]
[2025-05-13 07:37:01] INFO:     Waiting for application startup.
[2025-05-13 07:37:01] INFO:     Application startup complete.
[2025-05-13 07:37:01] INFO:     Uvicorn running on http://127.0.0.1:30335 (Press CTRL+C to quit)


[2025-05-13 07:37:02] INFO:     127.0.0.1:50760 - "GET /v1/models HTTP/1.1" 200 OK
[2025-05-13 07:37:02] INFO:     127.0.0.1:50776 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-13 07:37:02] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-13 07:37:03,279 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-13 07:37:03,297 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-13 07:37:03,304 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-13 07:37:03,317 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-05-13 07:37:03,736 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
2025-05-13 07:37:03,749 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-05-13 07:37:06,127 - INFO - flashinfer.jit: Loading JIT ops: quantization
2025-05-13 07:37:06,142 - INFO - flashinfer.jit: Finished loading JIT ops: quantization
[2025-05-13 07:37:06] INFO:     127.0.0.1:50790 - "POST /generate HTTP/1.1" 200 OK
[2025-05-13 07:37:06] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-05-13 07:37:07] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
2025-05-13 07:37:07,552 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-05-13 07:37:07,567 - INFO - flashinfer.jit: Finished loading JIT ops: cascade


[2025-05-13 07:37:07] INFO:     127.0.0.1:51504 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-05-13 07:37:07] Child process unexpectedly failed with an exit code 9. pid=978882
[2025-05-13 07:37:07] Child process unexpectedly failed with an exit code 9. pid=978808


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-13 07:37:14] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=32627, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=911449585, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gp

[2025-05-13 07:37:22] Casting torch.bfloat16 to torch.float16.


[2025-05-13 07:37:22] Casting torch.bfloat16 to torch.float16.
[2025-05-13 07:37:22] Attention backend not set. Use flashinfer backend by default.
[2025-05-13 07:37:22] Init torch distributed begin.


[2025-05-13 07:37:23] Init torch distributed ends. mem usage=0.00 GB
[2025-05-13 07:37:23] Load weight begin. avail mem=78.58 GB


[2025-05-13 07:37:24] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-05-13 07:37:41] INFO:     127.0.0.1:42514 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-13 07:37:41] INFO:     127.0.0.1:42528 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-13 07:37:41] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-13 07:37:41,866 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-13 07:37:41,883 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-13 07:37:41,890 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-13 07:37:41,901 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-05-13 07:37:42,285 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
2025-05-13 07:37:42,297 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-05-13 07:37:44,578 - INFO - flashinfer.jit: Loading JIT ops: quantization
2025-05-13 07:37:44,592 - INFO - flashinfer.jit: Finished loading JIT ops: quantization
[2025-05-13 07:37:44] INFO:     127.0.0.1:42534 - "POST /generate HTTP/1.1" 200 OK
[2025-05-13 07:37:44] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-05-13 07:37:46] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
2025-05-13 07:37:46,106 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-05-13 07:37:46,120 - INFO - flashinfer.jit: Finished loading JIT ops: cascade


[2025-05-13 07:37:46] INFO:     127.0.0.1:42550 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-05-13 07:37:46] Child process unexpectedly failed with an exit code 9. pid=979913
[2025-05-13 07:37:46] Child process unexpectedly failed with an exit code 9. pid=979847


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).